In [1]:
import pandas as pd
import requests
from splinter import Browser
import datetime as dt
from bs4 import BeautifulSoup
from sqlalchemy import create_engine
from config import username,password

In [2]:
# Splinter browser
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
# Read url
url = 'https://www.kroger.com/p/smithfield-thick-cut-naturally-hickory-smoked-bacon/0007080004125'
browser.visit(url)

In [4]:
# Browse and look for confirm and click thru
browser.find_by_text('Confirm').first.click()

In [5]:
# Scrape website and look for price of bacon
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
quotes = soup.find_all('mark',  class_="kds-Price-promotional")

In [6]:
# Once price is find, split and just collect the price
price=str(quotes[0]).split('>')[1].split('<')[0]
price=price.replace("$","")
print(price)

3.50


In [7]:
# Find the date
my_date=dt.date.today()


In [8]:
# Store the price and date into a dictionary
data_dict={"date":[my_date],
          "price":[price]}


In [9]:
# Print out data_dict
data_dict

{'date': [datetime.date(2020, 5, 30)], 'price': ['3.50']}

In [10]:
# Convert into a DataFrame
data_pd=pd.DataFrame(data_dict)

In [11]:
# Show DataFrame
data_pd

,date,price
0,2020-05-30,3.50


In [40]:
# Connect to PG Admin
rds_connection_string = f"{username}:{password}@localhost:5432/bacon_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [41]:
# Create table to PG Admin
data_pd.to_sql(name='kroger_price', con=engine, if_exists='append', index=False)